In [1]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()
# pyspark==2.4.7

'C:\\Spark\\spark-3.0.2-bin-hadoop2.7'

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [3]:
from pyspark.sql.functions import to_timestamp
from pyspark.sql import functions as F
from pyspark.sql.functions import sum,avg,max,min,mean
from pyspark.sql.functions import row_number,lit
from pyspark.sql.window import Window

In [4]:
conf = pyspark.SparkConf().setAppName('SparkApp').setMaster('local')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)

In [5]:
ADMISSIONS      = spark.read.option("header",True).csv('C:\Health data\ADMISSIONS.csv')
CHARTEVENTS     = spark.read.option("header",True).csv('C:\Health data\CHARTEVENTS.csv')
DIAGNOSES_ICD   = spark.read.option("header",True).csv('C:\Health data\DIAGNOSES_ICD.csv')
D_ICD_DIAGNOSES = spark.read.option("header",True).csv('C:\Health data\D_ICD_DIAGNOSES.csv')
ICUSTAYS        = spark.read.option("header",True).csv('C:\Health data\ICUSTAYS.csv')
LABEVENTS       = spark.read.option("header",True).csv('C:\Health data\LABEVENTS.csv')
OUTPUTEVENTS    = spark.read.option("header",True).csv('C:\Health data\OUTPUTEVENTS.csv')
PATIENTS        = spark.read.option("header",True).csv('C:\Health data\PATIENTS.csv')

In [6]:
####################### Basic ADMISSIONS and PATIENTS Data description ###############################
print( "how many ADMISSIONS recordes: ", ADMISSIONS.select('SUBJECT_ID').count() )
print( "how many patients: ", PATIENTS.select('SUBJECT_ID').distinct().count() )

patients_EMERGENCY = ADMISSIONS.filter(ADMISSIONS.ADMISSION_TYPE == 'EMERGENCY')
patients_Male = PATIENTS.filter(PATIENTS.GENDER == 'M')
patients_Female = PATIENTS.filter(PATIENTS.GENDER == 'F')

print("how many patients in EMERGENCY: ", patients_EMERGENCY.select('SUBJECT_ID').distinct().count())
print("how many Male patients: ", patients_Male.select('SUBJECT_ID').distinct().count())
print("how many Female patients: ", patients_Female.select('SUBJECT_ID').distinct().count())
print("how many mortality: ", PATIENTS.filter("EXPIRE_FLAG == '1'").select('SUBJECT_ID').distinct().count() )

patients_Non_EMERGENCY = ADMISSIONS.filter(ADMISSIONS.ADMISSION_TYPE != 'EMERGENCY').select('SUBJECT_ID').distinct()
mortality = PATIENTS.filter("EXPIRE_FLAG == '1'").select('SUBJECT_ID')

EMERGENCY_mortality = patients_EMERGENCY.select('SUBJECT_ID').intersect(mortality)
Non_EMERGENCY_mortality = patients_Non_EMERGENCY.intersect(mortality)
print("how many mortality in EMERGENCY: ", EMERGENCY_mortality.select('SUBJECT_ID').distinct().count() )
print("how many mortality in Non_EMERGENCY: ", Non_EMERGENCY_mortality.select('SUBJECT_ID').distinct().count() )

how many ADMISSIONS recordes:  58976
how many patients:  46520
how many patients in EMERGENCY:  32610
how many Male patients:  26121
how many Female patients:  20399
how many mortality:  15759
how many mortality in EMERGENCY:  14097
how many mortality in Non_EMERGENCY:  2699


In [7]:
####################### ICUSTAYS Data description ###############################
alive_ID     = PATIENTS.filter("EXPIRE_FLAG == '0'").select('SUBJECT_ID').distinct()
mortality_ID = PATIENTS.filter("EXPIRE_FLAG == '1'").select('SUBJECT_ID').distinct()
ICUSTAYS     = ICUSTAYS.withColumnRenamed("SUBJECT_ID", "ID")
alive_ID_ICU     = alive_ID.join( ICUSTAYS, alive_ID.SUBJECT_ID == ICUSTAYS.ID,"left" )
mortality_ID_ICU = mortality_ID.join( ICUSTAYS, mortality_ID.SUBJECT_ID == ICUSTAYS.ID,"left" )

alive_ID_ICU     = alive_ID_ICU.withColumn("LOS", alive_ID_ICU["LOS"].cast("double"))
mortality_ID_ICU = mortality_ID_ICU.withColumn("LOS", mortality_ID_ICU["LOS"].cast("double"))

print( "how mean ICU stay time of deceased patients: ", mortality_ID_ICU.groupBy("SUBJECT_ID").mean("LOS").agg({'avg(LOS)': 'mean'}).rdd.flatMap(list).collect()[0] )
print( "how max ICU stay time of deceased patients: ", mortality_ID_ICU.groupBy("SUBJECT_ID").max("LOS").agg({'max(LOS)': 'max'}).rdd.flatMap(list).collect()[0] )
print( "how min ICU stay time of deceased patients: ", mortality_ID_ICU.groupBy("SUBJECT_ID").min("LOS").agg({'min(LOS)': 'min'}).rdd.flatMap(list).collect()[0] )
print( "how mean ICU stay time of alive patients: ", alive_ID_ICU.groupBy("SUBJECT_ID").mean("LOS").agg({'avg(LOS)': 'mean'}).rdd.flatMap(list).collect()[0] )
print( "how max ICU stay time of alive patients: ", alive_ID_ICU.groupBy("SUBJECT_ID").max("LOS").agg({'max(LOS)': 'max'}).rdd.flatMap(list).collect()[0] )
print( "how min ICU stay time of alive patients: ", alive_ID_ICU.groupBy("SUBJECT_ID").min("LOS").agg({'min(LOS)': 'min'}).rdd.flatMap(list).collect()[0] )

how mean ICU stay time of deceased patients:  4.961010863396632
how max ICU stay time of deceased patients:  173.0725
how min ICU stay time of deceased patients:  0.0001
how mean ICU stay time of alive patients:  5.022757566913398
how max ICU stay time of alive patients:  171.6227
how min ICU stay time of alive patients:  0.0003
